# 存储数据

为了可以远程使用大部分网络爬虫，你还需要把采集到的数据存储起来。本章将介绍三种主要的数据管理方法，对绝大多数应用都适用。

- 如果你准备创建一个网站的后端服务或者创建自己的 API，那么可能都需要让爬虫把数据写入数据库。


- 如果你需要一个快速简单的方法收集网上的文档，然后存到你的硬盘里，那么可能需要创建一个文件流（file stream）来实现。


- 如果还要为偶然事件提个醒儿，或者每天定时收集当天累计的数据，就给自己发一封邮件吧！


抛开与网络数据采集的关系，大数据存储和与数据交互的能力，在新式的程序开发中也已经是重中之重了。



### 1　媒体文件

**在 Python 3.x 版本中，urllib.request.urlretrieve 可以根据文件的 URL 下载文件：** 

In [2]:
from urllib.request import urlretrieve,urlopen
from bs4 import BeautifulSoup

In [5]:
html = urlopen("http://www.pythonscraping.com")
bsObj = BeautifulSoup(html, "html.parser")
imageLocation = bsObj.find('a', {"id": "logo"}).find("img")["src"]
urlretrieve(imageLocation, "logo.jpg")

               

('logo.jpg', <http.client.HTTPMessage at 0x7af8a90>)

这段程序从 http://pythonscraping.com 下载 logo 图片，然后在程序运行的文件夹里保存为logo.jpg 文件。

如果你只需要下载一个文件，而且知道如何获取它，以及它的文件类型，这么做就可以了。但是大多数爬虫都不可能一天只下载一个文件。下面的程序会把 http://pythonscraping.com 主页上所有 src 属性的文件都下载下来：

In [9]:
import os
from urllib.request import urlretrieve
from urllib.request import urlopen
from bs4 import BeautifulSoup

downloadDirectory = 'downloaded'
baseUrl = 'http://pythonscraping.com'

def getAbsoluteURL(baseUrl, source):
    if source.startswith('http://www.'):
        url = 'http://{}'.format(source[11:])
    elif source.startswith('http://'):
        url = source
    elif source.startswith('www.'):
        url = source[4:]
        url = 'http://{}'.format(source)
    else:
        url = '{}/{}'.format(baseUrl, source)
    if baseUrl not in url:
        return None
    return url

def getDownloadPath(baseUrl, absoluteUrl, downloadDirectory):
    path = absoluteUrl.replace('www.', '')
    path = path.replace(baseUrl, '')
    path = downloadDirectory+path
    directory = os.path.dirname(path)

    if not os.path.exists(directory):
        os.makedirs(directory)

    return path

html = urlopen('http://www.pythonscraping.com')
bs = BeautifulSoup(html, 'html.parser')
downloadList = bs.findAll(src=True)

for download in downloadList:
    fileUrl = getAbsoluteURL(baseUrl, download['src'])
    if fileUrl is not None:
        print(fileUrl)

urlretrieve(fileUrl, getDownloadPath(baseUrl, fileUrl, downloadDirectory))

http://pythonscraping.com/misc/jquery.js?v=1.4.4
http://pythonscraping.com/misc/jquery.once.js?v=1.2
http://pythonscraping.com/misc/drupal.js?os2esm
http://pythonscraping.com/sites/all/themes/skeletontheme/js/jquery.mobilemenu.js?os2esm
http://pythonscraping.com/sites/all/modules/google_analytics/googleanalytics.js?os2esm
http://pythonscraping.com/sites/default/files/lrg_0.jpg
http://pythonscraping.com/img/lrg%20(1).jpg


('downloaded/img/lrg%20(1).jpg', <http.client.HTTPMessage at 0x788f0f0>)

### 程序运行注意事项
```
你知道从网上下载未知文件的那些警告吗？这个程序会把页面上所有的文件下载到你的硬盘里，
可能会包含一些 bash 脚本、.exe 文件，甚至可能是恶意软件（malware）。

如果你之前从没有运行过任何下载到电脑里的文件，电脑就是安全的吗？尤其是当你用管理员权
限运行这个程序时，你的电脑基本已经处于危险之中。如果你执行了网页上的一个文件，那个文
件把自己传送到了 ../../../../usr/bin/python 里面，会发生什么呢？等下一次你再运行 Python
程序时，你的电脑就可能会安装恶意软件。

这个程序只是为了演示；请不要随意运行它，因为这里没有对所有下载文件的类型进行检查，也不
应该用管理员权限运行它。记得经常备份重要的文件，不要在硬盘上存储敏感信息，小心驶得万年船。
```

**这个程序首先使用 Lambda 函数（第 2 章介绍过）选择首页上所有带 src 属性的标签。然后对 URL 链接进行清理和标准化，获得文件的绝对路径（而且去掉了外链）。最后，每个文件都会下载到程序所在文件夹的 downloaded 文件里.**

这里 Python 的 os 模块用来获取每个下载文件的目标文件夹，建立完整的路径。os 模块是Python 与操作系统进行交互的接口，它可以操作文件路径，创建目录，获取运行进程和环境变量的信息，以及其他系统相关的操作。

### 2    把数据存储到CSV

CSV（Comma-Separated Values，逗号分隔值）是存储表格数据的常用文件格式。Microsoft Excel 和很多应用都支持 CSV 格式，因为它很简洁。

和 Python 一样，CSV 里留白（whitespace）也是很重要的：每一行都用一个换行符分隔，列与列之间用逗号分隔（因此也叫“逗号分隔值”）。

**Python 的 csv 库可以非常简单地修改 CSV 文件，甚至从零开始创建一个 CSV 文件**：
```python
import csv

csvFile = open('test.csv', 'w+')
try:
    writer = csv.writer(csvFile)
    writer.writerow(('number', 'number plus 2', 'number times 2'))
    for i in range(10):
        writer.writerow( (i, i+2, i*2))
finally:
    csvFile.close()
```


In [11]:
import csv

csvFile = open('test.csv', "w+")
try:
    writer = csv.writer(csvFile)
    writer.writerow(('number', 'number plus 2', 'number times 2'))
    for i in range(10):
        writer.writerow((i, i+2, i**2))
finally:
    csvFile.close()

这里提个醒儿：Python 新建文件的机制考虑得非常周到（bullet-proof）。如果 ../files/test.csv不存在，Python 会自动创建文件（不会自动创建文件夹）。如果文件已经存在，Python 会用新的数据覆盖 test.csv 文件。

网络数据采集的一个常用功能就是获取HTML表格并写入CSV文件。维基百科的文本编辑器对比词条
（https://en.wikipedia.org/wiki/Comparison_of_text_editors） 中用了许多复杂的 HTML 表格，用到了颜色、链接、排序，以及其他在写入 CSV 文件之前需要忽略的HTML 元素。用 BeautifulSoup 和 get_text() 函数，你可以用十几行代码完成这件事：
```python
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://en.wikipedia.org/wiki/Comparison_of_text_editors')
bs = BeautifulSoup(html, 'html.parser')
#The main comparison table is currently the first table on the page
table = bs.findAll('table',{'class':'wikitable'})[0]
rows = table.findAll('tr')

csvFile = open('editors.csv', 'wt+')
writer = csv.writer(csvFile)
try:
    for row in rows:
        csvRow = []
        for cell in row.findAll(['td', 'th']):
            csvRow.append(cell.get_text())
            writer.writerow(csvRow)
finally:
    csvFile.close()
```

In [15]:
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://en.wikipedia.org/wiki/Comparison_of_text_editors')
bs = BeautifulSoup(html, 'html.parser')
#The main comparison table is currently the first table on the page
table = bs.findAll('table',{'class':'wikitable'})[0]
rows = table.findAll('tr')

csvFile = open('editors.csv', 'wt+')
writer = csv.writer(csvFile)
try:
    for row in rows:
        csvRow = []
        for cell in row.findAll(['td', 'th']):
            csvRow.append(cell.get_text())
            writer.writerow(csvRow)
finally:
    csvFile.close()

UnicodeEncodeError: 'gbk' codec can't encode character '\xf6' in position 15: illegal multibyte sequence

#### 实际工作中写此程序之前的注意事项
```
如果你有很多 HTML 表格，且每个都要转换成 CSV 文件，或者许多 HTML 表格都要汇总到一个 CSV 文件，那么把这个程序
整合到爬虫里以解决问题非常好。但是，如果你只需要做一次这种事情，那么更好的办法就是：复制粘贴。选择 HTML 表格
内容然后粘贴到 Excel 文件里，可以另存为 CSV 格式，不需要写代码就能搞定！
```

### 3 MySQL

MySQL是目前最受欢迎的开源关系型数据库管理系统。。对大多数应用来说，MySQL 都是不二选择。它是一种非常灵活、稳定、功能齐全的 DBMS，许多顶级的网站都在用它：YouTube、Twitter 和Facebook等。因为它受众广泛，免费，开箱即用，所以它也是网络数据采集项目中常用的数据库.

### 4 Email

与网页通过 HTTP 协议传输一样，邮件是通过 SMTP传输的。而且，和你用网络服务器的客户端（浏览器）处理那些通过HTTP 协议传输的网页一样，Email 服务器也有客户端，像 Sendmail、Postfix 和 Mailman等，都可以收发邮件。

虽然用 Python 发邮件很容易，但是需要你连接那些正在运行 SMTP 协议的服务器。在服务器或本地机器上设置 SMTP 客户端有点儿复杂，也超出了本书的介绍范围，但是有很多资料可以帮你解决问题，如果你用的是 Linux 或 Mac OS X 系统，参考资料会更丰富。

下面的代码运行的前提是你的电脑已经可以正常地运行一个 SMTP 客户端。（如果要调整代码用于远程 SMTP 客户端，请把 localhost 改成远程服务器地址。）

 Python 发一封邮件只要 9 行代码：
 ```python
import smtplib
from email.mime.text import MIMEText

msg = MIMEText('The body of the email is here')

msg['Subject'] = 'An Email Alert'
msg['From'] = 'ryan@pythonscraping.com'
msg['To'] = 'webmaster@pythonscraping.com'

s = smtplib.SMTP('localhost')
s.send_message(msg)
s.quit()
 ```

In [17]:
import smtplib
from email.mime.text import MIMEText

msg = MIMEText('The body of the email is here')

msg['Subject'] = 'An Email Alert'
msg['From'] = 'ryan@pythonscraping.com'
msg['To'] = 'webmaster@pythonscraping.com'

s = smtplib.SMTP('localhost')
s.send_message(msg)
s.quit()

ConnectionRefusedError: [WinError 10061] 由于目标计算机积极拒绝，无法连接。

#### Python 有两个包可以发送邮件：``smtplib`` 和 ``email``

Python 的 email 模块里包含了许多实用的邮件格式设置函数，可以用来创建邮件“包裹”。下面的示例中使用的 MIMEText 对象，为底层的MIME（Multipurpose Internet Mail Extensions，多用途互联网邮件扩展类型）协议传输创建了一封空邮件，最后通过高层的SMTP 协议发送出去。MIMEText 对象 msg 包括收发邮箱地址、邮件正文和主题，Python 通过它就可以创建一封格式正确的邮件。

smtplib 模块用来设置服务器连接的相关信息。就像 MySQL 服务器的连接一样，这个连接必须在用完之后及时关闭，以避免同时创建太多连接而浪费资源。

把这个简单的邮件程序封装成函数后，可以更方便地扩展和使用：
```python
import smtplib
from email.mime.text import MIMEText
from bs4 import BeautifulSoup
from urllib.request import urlopen
import time

def sendMail(subject, body):
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] ='christmas_alerts@pythonscraping.com'
    msg['To'] = 'ryan@pythonscraping.com'

    s = smtplib.SMTP('localhost')
    s.send_message(msg)
    s.quit()

bs = BeautifulSoup(urlopen('https://isitchristmas.com/'), 'html.parser')
while(bs.find('a', {'id':'answer'}).attrs['title'] == 'NO'):
    print('It is not Christmas yet.')
    time.sleep(3600)
    bs = BeautifulSoup(urlopen('https://isitchristmas.com/'), 'html.parser')
sendMail('It\'s Christmas!', 
         'According to http://itischristmas.com, it is Christmas!')
```

In [18]:
import smtplib
from email.mime.text import MIMEText
from bs4 import BeautifulSoup
from urllib.request import urlopen
import time

def sendMail(subject, body):
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] ='christmas_alerts@pythonscraping.com'
    msg['To'] = 'ryan@pythonscraping.com'

    s = smtplib.SMTP('localhost')
    s.send_message(msg)
    s.quit()

bs = BeautifulSoup(urlopen('https://isitchristmas.com/'), 'html.parser')
while(bs.find('a', {'id':'answer'}).attrs['title'] == 'NO'):
    print('It is not Christmas yet.')
    time.sleep(3600)
    bs = BeautifulSoup(urlopen('https://isitchristmas.com/'), 'html.parser')
sendMail('It\'s Christmas!', 
         'According to http://itischristmas.com, it is Christmas!')

ConnectionRefusedError: [WinError 10061] 由于目标计算机积极拒绝，无法连接。